In [2]:
import os
import sys
import json
from pathlib import Path
import shutil

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Tuple, Dict, Any, Optional

import sys
sys.path.append("..")

In [3]:
PATH_ROOT_DIR: str = ".."

INFO_TO_USE_FOR_CLUSTERING: List[str] = [
    'exception_message', 'essential_file_content']

PCA_COMPONENTS: int = 3

AI_MODEL: str = "llama-3.1-70b-versatile"  # or "gpt-4o-mini"

N_POSITIVE_EXAMPLES_TO_SHOW_LLM: int = 5

# Test run
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__14_48__qiskit")
# Run with 10k programs: v001.yaml
# program_bank/v001/2024_10_04__15_10__qiskit
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__15_10__qiskit")
# program_bank/v002/2024_10_07__17_03__qiskit
# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__qiskit")

# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__sample_errors")


# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__qiskit")
# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__sample_errors")

# inspected folders
# program_bank/v005/2024_10_22__22_32__qiskit
# program_bank/v007/2024_10_31__17_51__qiskit
# program_bank/v008/2024_11_07__15_05__qiskit
# program_bank/v009/2024_11_20__13_42__qiskit
# program_bank/v012/2024_12_16__23_03__qiskit
# program_bank/v014/2024_12_27__00_01__qiskit
# program_bank/v022/2025_01_23__12_48__qiskit
DIR_PROGRAM_BANK: str = os.path.join(
    "..", "program_bank", "v022", "2025_01_23__12_48__qiskit")
DIR_SAMPLE_ERRORS: str = os.path.join(
    "..", "program_bank", "v022", "2025_01_23__12_48__sample_errors")


PROMPT_CONTENT: str = "BOTH_ERROR_AND_PROGRAM"

In [4]:
# print the current folder
print(f"Current folder: {os.getcwd()}")

all_global_vars = dict(globals().copy())
# print all global variables
for k, v in all_global_vars.items():
    if k.isupper():
        print(f"{k}: {v}")

Current folder: /home/paltenmo/projects/crossplatform/notebooks
PATH_ROOT_DIR: ..
INFO_TO_USE_FOR_CLUSTERING: ['exception_message', 'essential_file_content']
PCA_COMPONENTS: 3
AI_MODEL: llama-3.1-70b-versatile
N_POSITIVE_EXAMPLES_TO_SHOW_LLM: 5
DIR_PROGRAM_BANK: ../program_bank/v022/2025_01_23__12_48__qiskit
DIR_SAMPLE_ERRORS: ../program_bank/v022/2025_01_23__12_48__sample_errors
PROMPT_CONTENT: BOTH_ERROR_AND_PROGRAM


In [5]:
import openai
import groq
PATH_GROQ_TOKEN = os.path.join(PATH_ROOT_DIR, "groq_token.txt")
GROQ_TOKEN = Path(PATH_GROQ_TOKEN).read_text().strip()

# Export the GROQ_TOKEN as GROQ_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_TOKEN

PATH_OPENAI_API_KEY = os.path.join(PATH_ROOT_DIR, "openai_token.txt")
OPENAI_API_KEY = Path(PATH_OPENAI_API_KEY).read_text().strip()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [6]:
# count generated programs in the program bank
count_generated_programs = len(list(Path(DIR_PROGRAM_BANK).rglob("*.py")))
print(f"Number of generated programs: {count_generated_programs}")

# count the number of errors
count_errors = len(list(Path(DIR_PROGRAM_BANK).rglob("*_error.json")))
print(f"Number of errors: {count_errors}")

Number of generated programs: 9026
Number of errors: 110739


## Read Files

In [7]:
from multiprocessing import Pool
import re


def read_json_file(file_path: str) -> Dict[str, Any]:
    with open(file_path, 'r') as file:
        data = json.load(file)
        data["error_file"] = os.path.basename(file_path)
        return data


def read_all_programs(dir_path: str) -> pd.DataFrame:
    json_files = [os.path.join(dir_path, f) for f in os.listdir(
        dir_path) if re.match(r'.*_error\.json$', f)]
    records = []

    with Pool() as pool:
        results = pool.map(read_json_file, json_files)
        records.extend(results)

    return pd.DataFrame(records)


# Use the function to read all programs
df_errors = read_all_programs(DIR_PROGRAM_BANK)

# augment the dataframe with the proram content


def read_file_content(file_name: str) -> str:
    file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
    with open(file_path, 'r') as file:
        return file.read()


# Use parallel apply to read the file content
with Pool() as pool:
    df_errors['file_content'] = pool.map(
        read_file_content, df_errors['current_file'])

# Alternatively, you can use apply without parallel processing
# df_errors['file_content'] = df_errors['current_file'].apply(read_file_content)

df_errors.head()
n_errors = len(df_errors)
print(f"Number of errors: {n_errors}")


# HELPER FUNCTIONS


def get_corresponding_program_file(path_error_file: str) -> str:
    # remove the fixed length prefix "_ba6b02_error.json"
    return path_error_file[:-18] + ".py"


def copy_errors_to_sample_folder(
        sample_errors_paths: List[str],
        dir_sample_errors: str):
    os.makedirs(dir_sample_errors, exist_ok=True)
    for file_path in sample_errors_paths:
        shutil.copy(file_path, dir_sample_errors)
        print(f"Copying {file_path} to {dir_sample_errors}")
        corresponding_program_file = get_corresponding_program_file(file_path)
        shutil.copy(corresponding_program_file, dir_sample_errors)
        print(f"Copying {corresponding_program_file} to {dir_sample_errors}")

Number of errors: 110739


## Explore the error messages

# Exclude Pennylane (errors)

In [10]:
df_errors['involved_functions'] = df_errors['involved_functions'].astype(
    str)
df_errors = df_errors[
    ~df_errors['involved_functions'].str.contains('pennylane')]
n_errors = len(df_errors)
print(f"Number of errors without PennyLane: {n_errors}")

Number of errors without PennyLane: 8220


# Minimize Sample

In [ ]:

# keep only errors with `The circuits are not equivalent` in the exception message
df_errors = df_errors[df_errors['exception_message'].str.contains(
    'The circuits are not equivalent')]


SAMPLE_SIZE = 10
actual_sample_size = min(SAMPLE_SIZE, len(df_errors))


# pick a sample of error files
sample_errors = df_errors.sample(actual_sample_size, random_state=42)

# get the paths of the error files
sample_errors_paths = [os.path.join(
    DIR_PROGRAM_BANK, f) for f in sample_errors['error_file']]


copy_errors_to_sample_folder(sample_errors_paths, DIR_SAMPLE_ERRORS)

Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_1511_600fe6_0353c4_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_1511_600fe6.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5923_1fb05d_394db9_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5923_1fb05d.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5585_30abba_76b749_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5585_30abba.py to ../program_bank/v009/2024_11_20__13_42__sample_err

In [11]:
# get the content of their error messages
sample_errors['exception_message']

12723    The circuits are not equivalent: /workspace/qi...
693      The circuits are not equivalent: /workspace/qi...
10624    The circuits are not equivalent: /workspace/qi...
3913     The circuits are not equivalent: /workspace/qi...
5149     The circuits are not equivalent: /workspace/qi...
11150    The circuits are not equivalent: /workspace/qi...
842      The circuits are not equivalent: /workspace/qi...
10721    The circuits are not equivalent: /workspace/qi...
11223    The circuits are not equivalent: /workspace/qi...
11601    The circuits are not equivalent: /workspace/qi...
Name: exception_message, dtype: object

In [30]:
import sys
sys.path.append("..")

## Triage All Files 
Including:
1. minimization
2. rerunning the code
3. running the analysis notebook

In [13]:
%load_ext autoreload
%autoreload 2

In [15]:
from analysis_and_reporting.triage import analyze_and_report

error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, f)
    for f in os.listdir(DIR_SAMPLE_ERRORS)
    if re.match(r'.*_error\.json$', f)
]

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# python -m analysis_and_reporting.triage --path_error_json program_bank/v007/2024_10_31__17_51__qiskit/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error.json --parent_report_folder reports/v008 --clue_message 'not equivalent'


# def analyze_and_report(
#         path_error_json: str, path_program: str, parent_report_folder: str,
#         clue_message: str, analysis_notebook: str)

for error_path in error_paths_in_sample_folder:
    try:
        analyze_and_report(
            path_error_json=error_path,
            path_program=get_corresponding_program_file(error_path),
            parent_report_folder=os.path.join("..", "reports", "v009"),
            clue_message='not equivalent',
            analysis_notebook='010_Triage_Not_Equivalence.ipynb'
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Skipping to the next error file")

Error paths in sample folder (10):
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4342_faa083_a812bf_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_6316_146402_6a36f0_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_6533_f0b204_dbc69c_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4756_28bbeb_444f6d_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_144_2ca34b_7b044a_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_680_6d896d_1fd766_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_5399_c4b732_f59bef_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_609_0c26d1_ee02f4_error.json
../program_bank/

[14:49:51] Created report folder: ../reports/v009/2024_11_21__14_49                                   ]8;id=600267;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=821025;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#131\131]8;;\

           Copied files to: ../reports/v009/2024_11_21__14_49                                         ]8;id=566959;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=440829;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#140\140]8;;\

[14:49:51] Involved functions:                                                             ]8;id=234626;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=101226;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=317805;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=556817;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

[15:27:12] Minimized file saved as:                                                        ]8;id=250509;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=510256;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#240\240]8;;\
           ../reports/v009/2024_11_21__15_23/qiskit_circuit_5q_10g_2997_458c2e_f885b3_erro                         
           r_min.py                                                                                                

Output: Saved the Pytket circuit to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_qiskit.qasm
Exported QASM files: [('/workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_pytket.qasm', 'export_to_qasm_with_pytket'), ('/workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_qiskit.qasm', 'export_to_qasm_with_qiskit')]
Log has been saved to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_da3a45_error.json



[15:27:18] Program qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min.py executed              ]8;id=134878;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py\docker_tooling.py]8;;\:]8;id=151265;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py#46\46]8;;\
           successfully.                                                                                           

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

[15:27:21] Analysis notebook executed: ../reports/v009/2024_11_21__15_23/analysis_output.ipynb        ]8;id=463065;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=328971;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#163\163]8;;\

# Inspect Crash Errors

In [7]:
# ! pip install datasets

In [8]:
df_errors.head()

,exception_message,stack_trace,current_file,involved_functions,timestamp,error_file,file_content
0,Could not import first circuit: <input>:5:1:\n...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_5570_0b246b.py,"[compare_qasm_via_qcec, export_to_qasm_with_bq...",1.735484e+09,qiskit_circuit_5q_10g_5570_0b246b_428131_error...,# Section: Prologue\nfrom qiskit import Quantu...
1,Could not import first circuit: <input>:5:1:\n...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_4199_4ec27d.py,"[compare_qasm_via_qcec, export_to_qasm_with_bq...",1.735443e+09,qiskit_circuit_5q_10g_4199_4ec27d_0fa4ad_error...,# Section: Prologue\nfrom qiskit import Quantu...
2,Could not import first circuit: <input>:5:1:\n...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_9849_7e6728.py,"[compare_qasm_via_qcec, export_to_qasm_with_bq...",1.735577e+09,qiskit_circuit_5q_10g_9849_7e6728_4473a2_error...,# Section: Prologue\nfrom qiskit import Quantu...
3,Classical register redeclared: c.,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_3148_6e588f.py,"[import_from_qasm_with_bqskit, export_to_qasm_...",1.735405e+09,qiskit_circuit_5q_10g_3148_6e588f_e5e5f9_error...,# Section: Prologue\nfrom qiskit import Quantu...
4,"""<input>:5,5: 'c' is already defined""","Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_9900_5a0bc9.py,"[import_from_qasm_with_qiskit, export_to_qasm_...",1.735579e+09,qiskit_circuit_5q_10g_9900_5a0bc9_140461_error...,# Section: Prologue\nfrom qiskit import Quantu...


In [9]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer

df_errors_no_dups = df_errors.copy()
df_errors_no_dups.drop_duplicates(subset=['exception_message'], inplace=True)

# # create a dataset from the error dataframe
# dataset = Dataset.from_pandas(df_errors_no_dups)

# # inspect the dataset
# dataset

# use deepseek-ai/DeepSeek-Coder-V2-Lite-Base
# from datasets import Dataset
# dataset = Dataset.from_dict({"a": [0, 1, 2]})
# dataset.map(lambda batch: {"b": batch["a"] * 2}, batched=True)  # new column with 6 elements: [0, 1, 2, 0, 1, 2]
# Load the tokenizer and model
# EMBEDDING_MODEL = "TinyLlama/TinyLlama_v1.1_math_code"
# EMBEDDING_MODEL = "all-MiniLM-L6-v2"
EMBEDDING_MODEL = "multi-qa-mpnet-base-cos-v1"
# EMBEDDING_MODEL = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"
# Load a pretrained Sentence Transformer model
model = SentenceTransformer(EMBEDDING_MODEL)

# Calculate embeddings for the exception_message column
df_errors_no_dups['exception_message_embeddings'] = list(
    model.encode(
        df_errors['exception_message'].tolist(),
        batch_size=32, show_progress_bar=True))

# Inspect the embeddings
df_errors_no_dups['exception_message_embeddings'].head()

Batches:   0%|          | 0/3461 [00:00<?, ?it/s]

ValueError: Length of values (110739) does not match length of index (49195)

In [10]:
# ! pip install fpsample

In [8]:
import fpsample
import numpy as np

# Extract the embeddings from the dataframe
embeddings = np.array(df_errors['exception_message_embeddings'].tolist())

# Sample 300 points using various FPS methods

# # Vanilla FPS
fps_samples_idx = fpsample.fps_sampling(embeddings, 300)

# Get the sampled data
sampled_data = df_errors.iloc[fps_samples_idx]
sampled_data.head()

,exception_message,stack_trace,current_file,involved_functions,timestamp,extra_info,error_file,file_content,exception_message_embeddings
17681,"""<input>:5,5: 'cr' is already defined""","Traceback (most recent call last):\n File ""/w...",0001597_a3b932.py,[import_from_qasm_with_pennylane],1.737647e+09,{'input_qasm_file': '0001597_a3b932_qc_bqskit....,0001597_a3b932_ec75b8_error.json,# Section: Prologue\nfrom qiskit import Quantu...,"[0.04345273, -0.06467503, 0.0014078164, 0.0097..."
41287,CUGate with nonzero phase,"Traceback (most recent call last):\n File ""/w...",0001570_a69df6.py,"[qiskit_to_tk, optimize_with_pytket]",1.737646e+09,None,0001570_a69df6_481819_error.json,# Section: Prologue\nfrom qiskit import Quantu...,"[0.008369477, -0.011292777, 0.016040538, -0.01..."
45415,Conversion of qiskit's dcx instruction is curr...,"Traceback (most recent call last):\n File ""/u...",0000203_16db48.py,"[qiskit_to_tk, optimize_with_pytket]",1.737635e+09,None,0000203_16db48_4871bc_error.json,# Section: Prologue\nfrom qiskit import Quantu...,"[-0.021260701, -0.039892245, -0.0064168647, -0..."
46007,'DaggerGate' object has no attribute '_qasm_name',"Traceback (most recent call last):\n File ""/w...",0000439_ff58df.py,[export_to_qasm_with_bqskit],1.737637e+09,None,0000439_ff58df_839528_error.json,# Section: Prologue\nfrom qiskit import Quantu...,"[0.03331826, 0.006505231, 0.012843889, 0.01305..."
45877,"Gate acts on 5 qubits, got 4 qubit variables.","Traceback (most recent call last):\n File ""/w...",0001104_1b7227.py,[import_from_qasm_with_bqskit],1.737642e+09,{'input_qasm_file': '0001104_1b7227_qc_RemoveR...,0001104_1b7227_06a9d2_error.json,# Section: Prologue\nfrom qiskit import Quantu...,"[0.0036534914, -0.042422894, -0.0018745466, 0...."


In [38]:
pd.set_option('display.max_colwidth', None)

keywords_to_highlight = [

    "parameters",

    # "The circuits are not equivalent"

    # BUG in BQSKIT
    # "already defined",
    # "already declared",

    # "satisf",
    # "support",
]


def highlight_keywords(row):
    for keyword in keywords_to_highlight:
        if keyword in row['exception_message']:
            return ['background-color: yellow'] * len(row)
    return [''] * len(row)


i = 0
size_chunk = 30
df = sampled_data[["exception_message", "error_file", "current_file"]
                  ].iloc[i*size_chunk:(i+1)*size_chunk]
# escape any < or > characters in exception_message
df['exception_message'] = df['exception_message'].str.replace(
    "<", "&lt;").str.replace(">", "&gt;")
df_styled = df.style.apply(highlight_keywords, axis=1)

# count the number of rows with the keyword
df_errors_no_dups = df_errors.copy()
df_errors_no_dups.drop_duplicates(subset=['exception_message'], inplace=True)


def display_prevalence_of(df, keywords):
    total = len(df)
    n_lines_with_keywords = df['exception_message'].apply(
        lambda x: any(keyword in x for keyword in keywords)).sum()
    perc_lines_with_keywords = n_lines_with_keywords / total * 100
    print(f"Percentage of error messages with {keywords}: ")
    print(f"{perc_lines_with_keywords:.2f}% ({n_lines_with_keywords}/{total})")


print("ALL ERRORs")
display_prevalence_of(df_errors.copy(), keywords=keywords_to_highlight)


print("UNIQUE ERRORs")
display_prevalence_of(df_errors_no_dups, keywords=keywords_to_highlight)


df_styled

ALL ERRORs
Percentage of error messages with ['parameters']: 
2.45% (1130/46080)
UNIQUE ERRORs
Percentage of error messages with ['parameters']: 
0.00% (1/20491)


,exception_message,error_file,current_file
17681,"""<input>:5,5: 'cr' is already defined""",0001597_a3b932_ec75b8_error.json,0001597_a3b932.py
41287,CUGate with nonzero phase,0001570_a69df6_481819_error.json,0001570_a69df6.py
45415,Conversion of qiskit's dcx instruction is currently unsupported by qiskit_to_tk. Consider using QuantumCircuit.decompose() before attempting conversion.,0000203_16db48_4871bc_error.json,0000203_16db48.py
46007,'DaggerGate' object has no attribute '_qasm_name',0000439_ff58df_839528_error.json,0000439_ff58df.py
45877,"Gate acts on 5 qubits, got 4 qubit variables.",0001104_1b7227_06a9d2_error.json,0001104_1b7227.py
44561,Cannot print command of type: CnZ,0001593_d97e09_c9b1d5_error.json,0001593_d97e09.py
5940,"The circuits are not equivalent: 0000288_ec8b6a_qc_RemoveRedundancies_pytket.qasm, 0000288_ec8b6a_qc_EulerAngleReduction_pytket.qasm",0000288_ec8b6a_9b5957_error.json,0000288_ec8b6a.py
27317,Cannot obtain matrix from op: Unitary1qBox,0001793_697371_afdb2e_error.json,0001793_697371.py
45646,"Unexpected token Token('$END', '') at line 1, column 1. Expected one of: * OPENQASM",0000153_647993_08f766_error.json,0000153_647993.py
42370,Classical register redeclared: cr.,0001503_b52b35_851a16_error.json,0001503_b52b35.py


In [8]:
from IPython.display import display, Markdown

# END empty qasm from bqskit >> due to daggergate crash
file_name = "0000153_647993_08f766_error.json"

# Cannot print command of type: CnZ >> unsupported error qasm pytket
file_name = "0001593_d97e09_c9b1d5_error.json"

# RuntimeError: Operation is not a gate: Unitary1qBox >> FullPeepholeOptimise
file_name = "0001793_697371_afdb2e_error.json"

file_path = os.path.join(DIR_PROGRAM_BANK, file_name)

data = json.loads(Path(file_path).read_text())

for key, value in data.items():
    display(Markdown(f"**{key}:**"))
    print(value)

**exception_message:**

Cannot obtain matrix from op: Unitary1qBox


**stack_trace:**

Traceback (most recent call last):
  File "/workspace/0001793_697371.py", line 520, in oracle_optimizer
    qasm_file = platform_specific_optimize_call(
  File "/workspace/0001793_697371.py", line 303, in optimize_with_pytket
    optimization_pass.apply(i_qc)
RuntimeError: Cannot obtain matrix from op: Unitary1qBox



**current_file:**

...

['qiskit_to_tk', 'optimize_with_pytket']


**timestamp:**

1737648259.106081


**extra_info:**

None


In [42]:
file_name = "0001793_697371_circuit.py"
file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
print(file_path)

content = Path(file_path).read_text()
display(Markdown(f"**{file_name}:**"))
print(content)

../program_bank/v022/2025_01_23__12_48__qiskit/0001793_697371_circuit.py


**0001793_697371_circuit.py:**

qr = QuantumRegister(11, name='qr')
cr = ClassicalRegister(11, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
# TIMESTAMP: 1737648255.013264

# Apply gate operations
# <START_GATES>
qc.p(1.095686, qr[8])
qc.rxx(2.607111, qr[5], qr[1])
qc.mcrz(4.172311, [qr[6], qr[7], qr[5]], qr[10])
qc.t(qr[5])
qc.iswap(qr[10], qr[5])
# <END_GATES>

# Section: Measurement
qc.measure(qr, cr)


In [43]:
# relevant errors
relevant_error_files_with_message = [
    # # probably QCEC error
    # ("qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json",
    #  "classic-controlled operation is not a StandardOperation"),
    # # interesting pair of divergent platforms
    # ("qiskit_circuit_5q_10g_5557_dc52a0_027c07_error.json",
    #  "The circuits are not equivalent")
    # cs gate not defined
    # ("qiskit_circuit_5q_10g_3811_4fe1d7_ac90d3_error.json",
    #  "cs' is not defined")
    # ('qiskit_circuit_5q_10g_3242_e7d1fd_6fa18e_error.json',
    #  'Try renaming the register')
    # ('qiskit_circuit_5q_10g_4249_6b20ab_f7dd09_error.json',
    #  'delay')
    ('0001793_697371_afdb2e_error.json',
     'Cannot obtain matrix from op: Unitary1qBox')
]

In [44]:
error_paths = [os.path.join(DIR_PROGRAM_BANK, f)
               for f, _ in relevant_error_files_with_message]

# copy the relevant errors to the sample folder
copy_errors_to_sample_folder(error_paths, DIR_SAMPLE_ERRORS)


def add_tags_to_program_content(program_content: str) -> str:
    """Add START_GATES and END_GATES tags to the program content.

    They are both new lines, so they can be used as delimiters.
    The "# <START_GATES>" is the the first line after "# Section: Circuit"
    The "# <END_GATES>" is the the first line before "# Helper Functions"
    """
    lines = program_content.split("\n")
    new_lines = []
    for i, line in enumerate(lines):
        if line.startswith("# Section: Circuit"):
            new_lines.append(line)
            new_lines.append("# <START_GATES>")
        elif line.startswith("# Helper Functions"):
            new_lines.append("# <END_GATES>")
            new_lines.append(line)
        else:
            new_lines.append(line)
    return "\n".join(new_lines)


# iterate over all the .py files in the sample folder
for file_name in os.listdir(DIR_SAMPLE_ERRORS):
    if file_name.endswith(".py"):
        file_path = os.path.join(DIR_SAMPLE_ERRORS, file_name)
        with open(file_path, 'r') as file:
            program_content = file.read()
        if "# <START_GATES>" not in program_content:
            program_content = add_tags_to_program_content(program_content)
        with open(file_path, 'w') as file:
            file.write(program_content)

Copying ../program_bank/v022/2025_01_23__12_48__qiskit/0001793_697371_afdb2e_error.json to ../program_bank/v022/2025_01_23__12_48__sample_errors
Copying ../program_bank/v022/2025_01_23__12_48__qiskit/0001793_697371.py to ../program_bank/v022/2025_01_23__12_48__sample_errors


In [45]:
from analysis_and_reporting.triage import analyze_and_report

# Extract the error paths from the relevant list
error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, error_file)
    for error_file, clue in relevant_error_files_with_message
]

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)


# Analyze and report for each relevant error file
for error_path, clue_message in relevant_error_files_with_message:
    try:
        analyze_and_report(
            path_error_json=os.path.join(DIR_SAMPLE_ERRORS, error_path),
            path_program=get_corresponding_program_file(
                os.path.join(DIR_SAMPLE_ERRORS, error_path)),
            parent_report_folder=os.path.join("..", "reports", "v022"),
            clue_message=clue_message,
            analysis_notebook='010_Triage_Not_Equivalence.ipynb')
    except Exception as e:
        print(f"Error: {e}")
        print("Skipping to the next error file")

Error paths in sample folder (1):
../program_bank/v022/2025_01_23__12_48__sample_errors/0001793_697371_afdb2e_error.json


[18:27:44] Created report folder: ../reports/v022/2025_01_23__18_27                                   ]8;id=296709;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=604161;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#132\132]8;;\

           Copied files to: ../reports/v022/2025_01_23__18_27                                         ]8;id=604796;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=379321;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#141\141]8;;\

[18:27:44] Involved functions:                                                             ]8;id=866495;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=518195;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - qiskit_to_tk                                                                ]8;id=121890;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=257466;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - optimize_with_pytket                                                        ]8;id=401018;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=730521;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

Output: Log has been saved to /workspace/to_execute_0830f4_error.json
Saved Optimized Pytket circuit (FullPeepholeOptimise) to /workspace/to_execute_qc_FullPeepholeOptimise_pytket.qasm
Saved Optimized Pytket circuit (PeepholeOptimise2Q) to /workspace/to_execute_qc_PeepholeOptimise2Q_pytket.qasm
Saved Optimized Pytket circuit (RemoveRedundancies) to /workspace/to_execute_qc_RemoveRedundancies_pytket.qasm
Saved Optimized Pytket circuit (EulerAngleReduction) to /workspace/to_execute_qc_EulerAngleReduction_pytket.qasm
Saved Optimized Pytket circuit (KAKDecomposition) to /workspace/to_execute_qc_KAKDecomposition_pytket.qasm
Saved Optimized Pytket circuit (CliffordPushThroughMeasures) to /workspace/to_execute_qc_CliffordPushThroughMeasures_pytket.qasm
Saved Optimized Pytket circuit (FlattenRegisters) to /workspace/to_execute_qc_FlattenRegisters_pytket.qasm
Saved Optimized Pytket circuit (PauliSimp) to /workspace/to_execute_qc_PauliSimp_pytket.qasm
Saved Optimized Pytket circuit (GreedyPauliS

[18:28:58] Minimized file saved as:                                                        ]8;id=135667;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=217667;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#240\240]8;;\
           ../reports/v022/2025_01_23__18_27/0001793_697371_afdb2e_error_min.py                                    

Output: Log has been saved to /workspace/0001793_697371_afdb2e_error_min_7989d9_error.json
Log has been saved to /workspace/0001793_697371_afdb2e_error_min_14928b_error.json
Saved Optimized Qiskit circuit (optimization level 0) to /workspace/0001793_697371_afdb2e_error_min_qc_opt_level_0_qiskit.qasm
Saved Optimized Qiskit circuit (optimization level 1) to /workspace/0001793_697371_afdb2e_error_min_qc_opt_level_1_qiskit.qasm
Saved Optimized Qiskit circuit (optimization level 2) to /workspace/0001793_697371_afdb2e_error_min_qc_opt_level_2_qiskit.qasm
Saved Optimized Qiskit circuit (optimization level 3) to /workspace/0001793_697371_afdb2e_error_min_qc_opt_level_3_qiskit.qasm



[18:29:04] Program 0001793_697371_afdb2e_error_min.py executed successfully.                   ]8;id=953931;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py\docker_tooling.py]8;;\:]8;id=48807;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py#49\49]8;;\

In [21]:
file_name = "qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json"
# file_name = "qiskit_circuit_5q_10g_2382_eaac27.py"
file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
print(file_path)

with open(file_path, 'r') as file:
    file_content = file.read()

print(file_content)

../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json
{
    "exception_message": "Underlying operation of classic-controlled operation is not a StandardOperation.\n   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b  c[0] == 1",
    "stack_trace": "Traceback (most recent call last):\n  File \"/workspace/qiskit_circuit_5q_10g_2382_eaac27.py\", line 566, in <module>\n    compare_call(a_file, b_file)\n  File \"/workspace/qiskit_circuit_5q_10g_2382_eaac27.py\", line 503, in compare_qasm_via_qcec\n    result = qcec.verify(\n  File \"/usr/local/lib/python3.10/site-packages/mqt/qcec/verify.py\", line 56, in verify\n    ecm = EquivalenceCheckingManager(circ1, circ2, configuration)\nValueError: Underlying operation of classic-controlled operation is not a StandardOperation.\n   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b  c[0] == 1\n",
    "current_file": "qiskit_circuit_5q_10g_2382_eaac27.py",
    "involved_functions": [
     

# Manual Analyze and Report

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from analysis_and_reporting.triage import analyze_and_report

error_path = os.path.join(
    "..", "reports", "v012", "01",
    "qiskit_circuit_5q_10g_7553_24d42f_76b26a_error.json")
program_path = os.path.join(
    "..", "reports", "v012", "01", "qiskit_circuit_5q_10g_7553_24d42f.py")

analyze_and_report(
    path_error_json=error_path,
    path_program=program_path,
    parent_report_folder=os.path.join("..", "reports", "v012"),
    clue_message="not equivalent",
    analysis_notebook='010_Triage_Not_Equivalence.ipynb')

[14:48:00] Created report folder: ../reports/v012/2024_12_18__14_48                                   ]8;id=733887;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=154665;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#132\132]8;;\

           Copied files to: ../reports/v012/2024_12_18__14_48                                         ]8;id=451624;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=273129;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#141\141]8;;\

[14:48:00] Involved functions:                                                             ]8;id=615598;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=66817;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=543724;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=825193;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=543021;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=863224;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

[14:48:21] Analysis notebook executed:                                                                ]8;id=163938;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=937492;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#216\216]8;;\
           ../reports/v012/2024_12_18__14_48/analysis_output_random_qc.ipynb                                       

           QASM pair:                                                                                 ]8;id=268127;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=977910;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#199\199]8;;\
           ('../reports/v012/2024_12_18__14_48/qiskit_circuit_5q_10g_7553_24d42f_76b26a_error_min_qc_              
           qiskit.qasm',                                                                                           
           '../reports/v012/2024_12_18__14_48/qiskit_circuit_5q_10g_7553_24d42f_76b26a_error_min_qc_p              
           ytket.qasm')                                                                                            

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

[14:48:23] Analysis notebook executed: ../reports/v012/2024_12_18__14_48/analysis_output_qc.ipynb     ]8;id=256588;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=887587;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#216\216]8;;\